# Prototyping

In this notebook, we will focus on prototyping and producing a simple demo model we developed in the previous chapter. We will use the production planning model as an example. Recall that the model is defined as follows:

$$
\begin{equation}
    \begin{aligned}
        & \underset{x}{\text{maximize}}
        & & \sum_{i \in I} (r_i - c_i) \cdot x_{i} \\
        & \text{subject to}
        & & \sum_{i \in I} c_i \cdot x_i \leq b, \\
        & & & \sum_{i \in I} x_i \leq s, \\
        & & & x_i \leq p_i \quad \forall i \in I, \\
        & & & x_i \geq 0 \quad \forall i \in I.
    \end{aligned}
\end{equation}
$$

where $I$ is the set of products, $x_i$ is the production quantity of product $i$, $r_i$ is the revenue of product $i$, $c_i$ is the cost of product $i$, $b$ is the budget, $s$ is the storage capacity, and $p_i$ is the production capacity of product $i$.

## Goal of prototyping

Prototyping is the process of creating a simple model that demonstrates the functionality of the model we developed. It is essentially a demo model that we can use to demonstrate the functionality of the model to the user.

Since the goal is to demonstrate the key **functionality** of the model, we will need to define what functionalities we want to demonstrate. For the production planning model, we want to demonstrate the following:

1. The user can upload an Excel file containing the data and the parameters.
2. The user can verify the data and the parameters.
3. The user can solve the model.
4. The user can view the solution.
5. The user can download the solution in Excel format.

Recall that the data we have is stored here:

In [1]:
EXCEL_URL = "https://docs.google.com/spreadsheets/d/e/2PACX-1vQzIHhg3mZq4eGXraXQZl07kduWMhwrnUqqs_gPT6qH_V1SWI3crMZMllxG6MX1sz3QJCFBjMt9tftr/pub?output=xlsx"

Please be sure to download the Excel file and ready to upload it. Let's start by installing the necessary packages.

In [2]:
!pip install -q pyomo
!pip install -q gradio
!apt-get install -y -qq glpk-utils

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.4/321.4 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 60.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.2 MB/s eta 0:00:00
Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 124565 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libcolamd2:amd64.


We will use `gradio` to create a simple UI for the demo. Let's import the necessary packages.

In [3]:
import numpy as np
import pandas as pd
import gradio as gr

## Creating Gradio Interface
Gradio is a Python library for building and sharing custom web interfaces for machine learning models. It is a great tool for prototyping and demonstrating the functionality of the model.

Suppose we want to create a simple UI that allows the user to upload an Excel file, visualize the data, and solve the model. Let's create a gradio interface that allows the user to do that.

Let's begin with creating the upload interface. We will use `gr.File` to create the upload interface.

In [4]:
with gr.Blocks() as demo:

    gr.Markdown("Upload an Excel file")

    file_input = gr.File(
        label="Upload an .xlsx file",
        file_types=[".xlsx"],
        type="filepath"
    )

What we did here is to initiate a `demo` object using `gr.Blocks()`. It is a container for the UI components.

As part of that container, we added
- a `gr.Markdown` component to display a title.
- a `gr.File` component to allow the user to upload an Excel file.
  
We can set what the helper text should read, the file types we want to accept, the type of the file, etc.

Then, we can run the demo using `demo.launch()`.

In [5]:
demo.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6dc482c11eec745d26.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


By default, the demo will be launched in localhost and generate a public link. We can set `share=False` to disable the public link.

In [6]:
demo.launch(share=False)

Rerunning server... use `close()` to stop if you need to change `launch()` parameters.
----
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

Let's create a more complete interface by adding the following components:
- a `gr.Dataframe` component to display the data.
- a `gr.Button` component to allow the user to click and trigger various events

In [7]:
with gr.Blocks() as demo:
    gr.Markdown("Upload an Excel file")
    file_input = gr.File(
        label="Upload an .xlsx file",
        file_types=[".xlsx"],
        type="filepath"
    )

    gr.Markdown("Verify the data")
    data_output_df = gr.Dataframe(
        label="Data",
        wrap=True,
    )
    data_output_params = gr.Dataframe(
        label="Data",
        wrap=True,
    )

    gr.Button("Solve the model")
    gr.Markdown("Solution")
    solution_output = gr.Dataframe(
        label="Solution",
        wrap=True,
    )
    gr.Button("Download the solution")

Let's see how it looks like now.

In [8]:
demo.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b24a4c3f2934c800d5.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Great! Let's now add a function to load the data after the upload is complete. We add a callback function to the `file_input` component.

In [9]:
def load_data(file_input):
    df = pd.read_excel(file_input, sheet_name='data', index_col=0)
    params = pd.read_excel(file_input, sheet_name='params')
    return df, params

Let's add this into our interface.

In [10]:
with gr.Blocks() as demo:
    gr.Markdown("Upload an Excel file")
    file_input = gr.File(
        label="Upload an .xlsx file",
        file_types=[".xlsx"],
        type="filepath"
    )
    gr.Markdown("Verify the data")
    data_output_df = gr.Dataframe(
        label="Data",
        wrap=True,
    )
    data_output_params = gr.Dataframe(
        label="Data",
        wrap=True,
    )

    file_input.change(load_data, inputs=file_input, outputs=[data_output_df, data_output_params])

demo.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://430743ee67c282e9a7.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Similar to the V&V notebook, let's now wrap the model creation in a function that takes the data and parameters as inputs and returns the model object. Let's start with the function to format the data and parameters to be used in the model.

In [11]:
def preprocess_data(df_data, df_params):
    df = df_data.to_dict()
    params = df_params.to_dict(orient='list')
    params = {name_: value_ for name_, value_ in zip(params["name"], params["val"])}
    return df, params

In [12]:
import pyomo.environ as pyo

model = pyo.ConcreteModel()


#define the set of products
model.I = pyo.Set(initialize=[1, 2, 3])

model.pprint()

1 Set Declarations
    I : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {1, 2, 3}

1 Declarations: I


Let's now create the model-building function.

In [13]:
def build_production_planning_model(df, params, initial_values=0.0):
    #initiate the model
    model = pyo.ConcreteModel()


    #define the set of products
    model.I = pyo.Set(initialize=[0, 1, 2])


    #define the decision variables
    model.x = pyo.Var(model.I, domain=pyo.Reals, initialize=initial_values)


    #define the parameters
    model.r = pyo.Param(model.I, initialize=df['revenue'])
    model.c = pyo.Param(model.I, initialize=df['cost'])
    model.p = pyo.Param(model.I, initialize=df['production_capacity'])
    model.b = pyo.Param(initialize=params['budget'])
    model.s = pyo.Param(initialize=params['capacity'])

    #define useful expressions
    model.total_revenue = pyo.Expression(rule=lambda model: sum(model.r[i] * model.x[i] for i in model.I))
    model.total_cost = pyo.Expression(rule=lambda model: sum(model.c[i] * model.x[i] for i in model.I))


    #define the objective function
    def f(model):
        return model.total_revenue - model.total_cost

    model.obj = pyo.Objective(rule=f, sense=pyo.maximize)


    #define the constraints
    def budget_constraint(model):
        return sum(model.c[i] * model.x[i] for i in model.I) <= model.b
    model.budget_constraint = pyo.Constraint(rule=budget_constraint)

    def storage_capacity(model):
        return sum(model.x[i] for i in model.I) <= model.s
    model.storage_capacity = pyo.Constraint(rule=storage_capacity)

    def production_capacity(model, i):
        return model.x[i] <= model.p[i]
    model.production_capacity = pyo.Constraint(model.I, rule=production_capacity)

    def nonnegative_domain(model, i):
        return model.x[i] >= 0
    model.nonnegative_domain = pyo.Constraint(model.I, rule=nonnegative_domain)

    return model

Finally, let's solve the model. We will use the `pyomo` package to solve the model and extract the solution into a DataFrame.

In [14]:
def solve_model(model):
    solver = pyo.SolverFactory('glpk', executable='/usr/bin/glpsol')
    result = solver.solve(model, tee=True)

    if result.solver.termination_condition == pyo.TerminationCondition.optimal:
        x_opt_dict = {"i" : {i: i for i in model.I},
                      "x_opt" : {i: pyo.value(model.x[i]) for i in model.I}
                      }

        df = pd.DataFrame(x_opt_dict)
        file = df.to_excel("solution.xlsx")
        return  df, "solution.xlsx"
    else:
        #return a DataFrame with empty columns
        return pd.DataFrame(columns=model.I), None



Let's now create a function to tie all the components together and display the solution when the user clicks the "Solve the model" button.

In [15]:
def solve_and_display_solution(df_data, df_params):
    df, params = preprocess_data(df_data, df_params)
    model = build_production_planning_model(df, params, initial_values=0.0)
    solution, file = solve_model(model)
    return solution, file

Let's integrate the components together and run the demo.

In [16]:
with gr.Blocks() as demo:
    gr.Markdown("Upload an Excel file")
    file_input = gr.File(
        label="Upload an .xlsx file",
        file_types=[".xlsx"],
        type="filepath"
    )

    gr.Markdown("Verify the data")
    data_output_df = gr.Dataframe(
        label="Data",
        wrap=True,
    )
    data_output_params = gr.Dataframe(
        label="Data",
        wrap=True,
    )

    solve_button = gr.Button("Solve the model")
    gr.Markdown("Solution")
    solution_output = gr.Dataframe(
        label="Solution",
        wrap=True,
    )
    file_output = gr.File(label="Download Solution")

    file_input.change(load_data, inputs=file_input, outputs=[data_output_df, data_output_params])
    solve_button.click(solve_and_display_solution, inputs=[data_output_df, data_output_params], outputs=[solution_output, file_output])

demo.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2cdc67cac8b5c1d5a0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## Conclusion

In this notebook, we have demonstrated how to create a simple Gradio interface to demo key functionalities in the model we develop (upload an Excel file, verify the data, solve the model, and download the solution).